# Convolutions

$$
(f∗g)(i,j) = ∑_a ∑_b f(a,b)g(i−a,j−b)
$$

In [ ]:
import torch
import torch.nn as nn

In [ ]:
def conv2d(image, kernel):
    H, W = list(image.size())
    M, N = list(kernel.size())

    out = torch.zeros(H-M+1, W-N+1, dtype=torch.float32)
    for i in range(H-M+1):
        for j in range(W-N+1):
            out[i,j]= torch.sum(image[i:i+M,j:j+N]*kernel)
    return out

In [ ]:
conv_layer = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5)
print(conv_layer)

In [ ]:
input_img = torch.rand(1,3,7,7)
layer = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=2, padding=1)
out = layer(input_img)
print(out.shape)

In [ ]:
input_img = torch.rand(1,3,8,8)
layer = nn.MaxPool2d(kernel_size=2, stride=2)
out = layer(input_img)
print(out.shape)

In [ ]:
import torch
import torch.nn as nn

seed = 172
torch.manual_seed(seed)


class SkipConnection(nn.Module):

    def __init__(self):
        super(SkipConnection, self).__init__()
        self.conv_layer1 = nn.Conv2d(3, 6, 2, stride=2, padding=2)
        self.relu = nn.ReLU(inplace=True)
        self.conv_layer2 = nn.Conv2d(6, 3, 2, stride=2, padding=2)
        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, input: torch.FloatTensor) -> torch.FloatTensor:
        x = self.conv_layer1(input)
        x = self.relu(x)
        x = self.conv_layer2(x)
        x = self.relu2(x)
        return x + input